# Récupération des Etablissements d'enseignement supérieurs




En préalable je lance un container docker avec un volume exposé

```
$ docker run -d --publish=7474:7474 \
    --publish=7687:7687  \
    --volume=$HOME/neo4j/labos/data:/data  \
    --env=NEO4J_AUTH=none  \
    neo4j
```

Je vérifie que le serveur fonctionne bien : 
    
```
$ docker container ps 
CONTAINER ID        IMAGE               COMMAND                  CREATED             STATUS              PORTS                                                      NAMES
47c79c7c1871        neo4j               "/sbin/tini -g -- /d…"   About an hour ago   Up About an hour    0.0.0.0:7474->7474/tcp, 7473/tcp, 0.0.0.0:7687->7687/tcp   musing_vaughan

```

In [1]:
import pandas as pd
import io
import requests

from py2neo import Graph, Node, Relationship
from py2neo.database import Schema

graph = Graph()

## Récupération des données

In [2]:
structure_url = 'https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-structures-recherche-publiques-actives/download/?format=json'
structures = pd.read_json(io.StringIO(requests.get(structure_url).content.decode('utf-8')))

In [3]:
_id = lambda p : p
_to_int = lambda p : int(p) if p != '' else None
s_ = lambda s,name : s.get(name,'').split(';')
same_or_none = lambda x : x if x else None

def create_struct(structures, structure):
    scalars_properties = {'numero_national_de_structure': _id, 
                          'libelle': _id,
                          'sigle': _id,
                          'annee_de_creation': _to_int
                         }
    as_map = {}
    for key, fn in scalars_properties.items():
        value = structure.get(key)
        as_map[key] = fn(value)
    structures.append(as_map)

def create_tutelles(tutelles, structure, relations):
    
    tutelle_list = zip(s_(structure,'tutelles'), 
                       s_(structure,'code_de_type_de_tutelle'), 
                       s_(structure,'sigles_des_tutelles'),
                       s_(structure,'uai_des_tutelles'))
    
    
    for position, tutelle in enumerate(tutelle_list):
            label, code_type, sigle, uai = tutelle
            as_map = {}
            if label: 
                if label in tutelles:
                    as_map = tutelles[label]
                else :
                    as_map = {}
                    tutelles[label] = as_map
                if not label:
                    print(tutelle)
                as_map['label'] = same_or_none(label)
                as_map['type'] = same_or_none(code_type)
                as_map['sigle'] = same_or_none(sigle)
                as_map['uai'] = same_or_none(uai)  
                
                num_national = structure.get('numero_national_de_structure')
                relations.append( { 'nns': num_national, 'uai': uai, 'position': position})


In [13]:
def create_domain(domains, structure, relations) :
    domain_list = zip(s_(structure,'code_domaine_scientifique'), 
                       s_(structure,'domaine_scientifique'))
    for position, domain in enumerate(domain_list):
        code, label = domain
        as_map = {}
        if code : 
            as_map = domains.get(code, {})
            as_map['code'] = code
            as_map['label'] = label
            if code not in domains:
                domains[code] = as_map
                
            num_national = structure.get('numero_national_de_structure')
            relations.append( { 'nns': num_national, 'code': code, 'position': position})


In [15]:
structures_list = []
tutelles = {}
relations = []
domains = {}
domains_relation = []
for structure in structures['fields']:
    create_struct(structures_list, structure)
    create_tutelles(tutelles, structure, relations)
    create_domain(domains,structure, domains_relation)

structure_df = pd.DataFrame(structures_list)
tutelles_df = pd.DataFrame(list(tutelles.values()))
relations_df = pd.DataFrame(relations)  
domains_pd = pd.DataFrame(list(domains.values()))
domains_relation_pd = pd.DataFrame(domains_relation)
structure_df.to_csv('/tmp/structures.csv', sep=";")
tutelles_df.to_csv('/tmp/tutelles.csv', sep=";")
relations_df.to_csv('/tmp/relation_structure_tutelles.csv', sep=';')
domains_pd.to_csv('/tmp/domains.csv', sep=';')
domains_relation_pd.to_csv('/tmp/domain_relations.csv', sep=";")


Si vous exécuter sur votre poste, allez copier les fichiers sur le docker.

Dans mon cas, mon container _neo4j_ s'appelle _musing_vaughan_.

```bash
cd /tmp
docker container cp structures.csv musing_vaughan:/var/lib/neo4j/import
docker container cp tutelles.csv musing_vaughan:/var/lib/neo4j/import
docker container cp relation_structure_tutelles.csv musing_vaughan:/var/lib/neo4j/import

```

### Préparation de l'import

```cypher
CREATE INDEX ON :Structure(nns)
```

```cypher
CREATE INDEX ON :Tutelle(uai)
```

### Chargement des tutelles

```cypher
LOAD CSV WITH HEADERS FROM 'file:///tutelles.csv' as row FIELDTERMINATOR ';'
CREATE (:Tutelle { uai: row.uai, type: row.type, label: row.label, sigle: row.sigle})
```

 Added 276 labels, created 276 nodes, set 853 properties, completed after 564 ms.

### Chargement des structures

```cypher
USING PERIODIC COMMIT 500
LOAD CSV WITH HEADERS FROM "file:///structures.csv" as row FIELDTERMINATOR ';'
CREATE (:Structure {nns: row.numero_national_de_structure, sigle: row.sigle, label: row.label, creation_year: toInteger(row.annee_de_creation)})
```
 Added 3757 labels, created 3757 nodes, set 10364 properties, completed after 2253 ms.

### Etablissement des relations

```cypher
LOAD CSV WITH HEADERS FROM "file:///relation_structure_tutelles.csv" as row FIELDTERMINATOR ';'
MATCH (s:Structure {nns: row.nns}), (t:Tutelle {uai: row.uai})
CREATE (s)-[:ATTACHED_TO {position: toInteger(row.position)}]->(t)
```
 Set 7974 properties, created 7974 relationships, completed after 1484 ms.

### Mise à jour en masse

```cypher
LOAD CSV WITH HEADERS FROM "file:///structures.csv" as row FIELDTERMINATOR ';'
MATCH (s:Structure {nns: row.numero_national_de_structure})
SET s.label = row.libelle
```
 Set 3801 properties, completed after 645 ms.

### Les domaines

```cypher
LOAD CSV WITH HEADERS FROM "file:///domains.csv" as row FIELDTERMINATOR ';' MERGE (d:Domain { code: row.code, label: row.label})
```

## Quelques recherches

### Nombre de structures par tutelles

```cypher
MATCH (t:Tutelle)-[r:ATTACHED_TO]-(s:Structure)
WITH t, count(s) as nb
RETURN t.label as NomTutelle, nb as NombreStructure ORDER BY nb DESC
```


In [9]:
b

,NomTutelle,NombreStructure
0,Centre national de la recherche scientifique,1593
1,Institut national de la santé et de la recherc...,502
2,Institut national de la recherche agronomique,328
3,Institut national de recherche en informatique...,296
4,Institut Pasteur,197
5,Sorbonne Université,188
6,Commissariat à l'énergie atomique et aux énerg...,167
7,Aix-Marseille université,140
8,Université de Grenoble Alpes,140
9,Université Paris-Sud,109


In [11]:
graph.run('''MATCH (t:Tutelle)-[r:ATTACHED_TO]-(s:Structure)
WHERE r.position = 0
WITH t, count(s) as nb
RETURN t.label as NomTutelle, nb as NombreStructure ORDER BY nb DESC''').to_data_frame().head(20)

,NomTutelle,NombreStructure
0,Centre national de la recherche scientifique,651
1,Institut national de recherche en informatique...,236
2,Institut national de la recherche agronomique,174
3,Institut national de la santé et de la recherc...,159
4,Institut Pasteur,135
5,Sorbonne Université,99
6,Commissariat à l'énergie atomique et aux énerg...,84
7,Aix-Marseille université,66
8,Université de Strasbourg,63
9,Université de Grenoble Alpes,56
